Código responsável por cálcular o RBR de algumas imagens com nebulosidade COM O SOL ENCOBERTO

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


RBR PARA IMAGENS DE MARÇO A CADA 1 MIN

In [ ]:
import cv2
import os
import numpy as np

def calcular_razao_rbr(imagem_path, diretorio_saida):
    # Leitura da imagem
    imagem = cv2.imread(imagem_path)
    if imagem is None:
        print(f"Erro: Não foi possível ler a imagem em {imagem_path}")
        return None

    # Inverte a imagem horizontalmente
    imagem = cv2.flip(imagem, 1)

    # Ajuste para evitar divisão por zero no canal azul
    b_channel = imagem[:, :, 0]
    b_channel[b_channel == 0] = 1

    # Calcula a razão RBR
    r_channel = imagem[:, :, 2]
    rbr_ratio = r_channel / b_channel

    # Cria uma imagem binária baseada na razão RBR
    imagem_binaria = np.where(rbr_ratio <= 0.70, 0, 255)

    return imagem_binaria

# Diretório onde estão as pastas dos grupos
diretorio_principal = '2020/2020/1_min_rbr/Marco'
pastas = ['grupo_1', 'grupo_2', 'grupo_3', 'grupo_4', 'grupo_5', 'grupo_6', 'grupo_7', 'grupo_8', 'grupo_9', 'grupo_10']

# Para cada pasta de grupo
for pasta_grupo in pastas:
    diretorio_grupo = os.path.join(diretorio_principal, pasta_grupo)
    diretorio_saida_grupo = os.path.join(diretorio_grupo, f'{pasta_grupo}')

    # Cria o diretório de saída para as imagens binárias se não existir
    if not os.path.exists(diretorio_saida_grupo):
        os.makedirs(diretorio_saida_grupo)

    # Lista todos os arquivos no diretório do grupo
    for nome_arquivo in os.listdir(diretorio_grupo):
        # Verifica se é um arquivo de imagem .jpg
        if nome_arquivo.endswith('.jpg'):
            imagem_path = os.path.join(diretorio_grupo, nome_arquivo)
            # Calcula a imagem binária
            imagem_binaria = calcular_razao_rbr(imagem_path, diretorio_saida_grupo)
            if imagem_binaria is not None:
                # Salva a imagem binária como imagem
                nome_arquivo_saida = nome_arquivo.replace('.jpg', '_binaria.jpg')
                caminho_saida = os.path.join(diretorio_saida_grupo, nome_arquivo_saida)
                cv2.imwrite(caminho_saida, imagem_binaria)  # Multiplica por 255 para manter a escala de 0 a 255
                print(f"Imagem binária de saída salva como: {caminho_saida}")

print("Processo concluído!")


Aplicando farbeneck para as imagens binárias

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Diretório onde estão as pastas dos grupos
diretorio_principal = '2020/2020/1_min_rbr/Marco'
pastas = ['grupo_1', 'grupo_2', 'grupo_3', 'grupo_4', 'grupo_5', 'grupo_6', 'grupo_7', 'grupo_8', 'grupo_9', 'grupo_10']
#grupo = ['grupo_1_binaria', 'grupo_2_binaria', 'grupo_3_binaria', 'grupo_4_binaria', 'grupo_5_binaria', 'grupo_6_binaria', 'grupo_7_binaria', 'grupo_8_binaria', 'grupo_9_binaria', 'grupo_10_binaria']

# Função para calcular e plotar o fluxo óptico para uma imagem de grupo
def plotar_fluxo_optico(diretorio_grupo):
    # Lista os arquivos na pasta do grupo
    arquivos_grupo = os.listdir(diretorio_grupo)
    # Seleciona os dois primeiros arquivos de imagem encontrados
    imagens_grupo = [os.path.join(diretorio_grupo, arquivo) for arquivo in arquivos_grupo[:2] if arquivo.endswith('.jpg')]
    if len(imagens_grupo) < 2:
        print(f"Não foi possível encontrar duas imagens na pasta {diretorio_grupo}.")
        return
    # Carrega as imagens
    processed_image0 = cv2.imread(imagens_grupo[0])
    processed_image1 = cv2.imread(imagens_grupo[1])

    # Converte para escala de cinza
    gray_image0 = cv2.cvtColor(processed_image0, cv2.COLOR_BGR2GRAY)
    gray_image1 = cv2.cvtColor(processed_image1, cv2.COLOR_BGR2GRAY)

    # Calcula o fluxo óptico entre as imagens apenas nas áreas brancas da imagem binária
    flow0_1 = cv2.calcOpticalFlowFarneback(prev=gray_image0, next=gray_image1, flow=None, pyr_scale=0.5, levels=2, winsize=30, iterations=5, poly_n=5, poly_sigma=1.0, flags=0)

    # Deslocamento da imagem para criar a imagem do futuro
    new_image = np.zeros_like(gray_image1)
    for i in range(gray_image1.shape[0]):
        for j in range(gray_image1.shape[1]):
            if gray_image1[i, j] == 255:  # Verifica se o pixel é branco
                dx = flow0_1[i, j, 0]
                dy = flow0_1[i, j, 1]
                new_i = int(i + dy)
                new_j = int(j + dx)
                if 0 <= new_i < gray_image1.shape[0] and 0 <= new_j < gray_image1.shape[1]:
                    new_image[new_i, new_j] = 255  # Define o pixel como branco na nova imagem

    # Aplicar filtro gaussiano na imagem do futuro
    sigma = 2  # Ajuste o valor de sigma conforme necessário
    new_image_suavizada = cv2.medianBlur(new_image, ksize=9)

    # Construir o caminho completo para salvar o arquivo
    nome_arquivo = os.path.join(diretorio_grupo, os.path.basename(diretorio_grupo) + '_suavizada.jpg')

    # Salvar a imagem suavizada
    cv2.imwrite(nome_arquivo, new_image_suavizada)

    # Plotar as imagens
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)

    plt.imshow(new_image, cmap='gray')
    plt.title('Imagem do Futuro')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(new_image_suavizada, cmap='gray')
    plt.title('Imagem do Futuro Suavizada')
    plt.axis('off')

    plt.show()

# Itera sobre as pastas dos grupos e aplica Farneback para cada uma
for pasta_grupo in pastas:
    diretorio_grupo = os.path.join(diretorio_principal, pasta_grupo)
    plotar_fluxo_optico(diretorio_grupo)

print("Processo concluído!")


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Configurações de parâmetros para o Farneback
parametros = [
    (0.5, 5, 15, 5, 5, 1.0, 0),
    (0.5, 7, 25, 3, 5, 1.0, 0),
    (0.5, 2, 30, 5, 5, 1.0, 0),
    (0.5, 2, 40, 5, 5, 1.1, 0)
]

# Diretório onde estão as pastas dos grupos
diretorio_principal = '2020/2020/1_min_rbr/Marco'
pastas = [f'grupo_{i}' for i in range(1, 11)]

# Função para calcular e plotar o fluxo óptico para uma imagem de grupo
def plotar_fluxo_optico(diretorio_grupo):
    arquivos_grupo = os.listdir(diretorio_grupo)
    imagens_grupo = [os.path.join(diretorio_grupo, arquivo) for arquivo in arquivos_grupo if arquivo.endswith('.jpg')]
    if len(imagens_grupo) < 2:
        print(f"Não foi possível encontrar duas imagens na pasta {diretorio_grupo}.")
        return

    imagens_grupo.sort()  # Garantir ordem correta das imagens
    img1 = cv2.imread(imagens_grupo[0])
    img2 = cv2.imread(imagens_grupo[1])

    # Converte para escala de cinza
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))
    #fig.suptitle(f'Análise de Fluxo Óptico para {os.path.basename(diretorio_grupo)}')


    for idx, params in enumerate(parametros):
        flow = cv2.calcOpticalFlowFarneback(img1, img2, None, *params)
        new_image = np.zeros_like(img2)
        for i in range(img2.shape[0]):
            for j in range(img2.shape[1]):
                dx = int(flow[i, j, 0])
                dy = int(flow[i, j, 1])
                new_i, new_j = i + dy, j + dx
                if 0 <= new_i < img2.shape[0] and 0 <= new_j < img2.shape[1]:
                    new_image[new_i, new_j] = img2[i, j]

        ax = axs[idx // 2, idx % 2]
        ax.imshow(new_image, cmap='gray')
        ax.set_title(f'Parâmetros: levels={params[1]}, winsize={params[2]}, interations={params[3]}')
        ax.axis('off')

# Iterar sobre as pastas dos grupos e aplicar Farneback para cada uma
for pasta_grupo in pastas:
    diretorio_grupo = os.path.join(diretorio_principal, pasta_grupo)
    plotar_fluxo_optico(diretorio_grupo)

print("Processo concluído!")


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Diretório onde estão as pastas dos grupos
diretorio_principal = '2020/2020/1_min_rbr/Marco'
pastas = ['grupo_1', 'grupo_2', 'grupo_3', 'grupo_4', 'grupo_5', 'grupo_6', 'grupo_7', 'grupo_8', 'grupo_9', 'grupo_10']

# Valores de ksize para testar
ksizes = [3, 5, 7, 9]

# Função para calcular e plotar o fluxo óptico e suavização para uma imagem de grupo
def plotar_fluxo_optico(diretorio_grupo):
    arquivos_grupo = os.listdir(diretorio_grupo)
    imagens_grupo = [os.path.join(diretorio_grupo, arquivo) for arquivo in arquivos_grupo if arquivo.endswith('.jpg')]
    if len(imagens_grupo) < 2:
        print(f"Não foi possível encontrar duas imagens na pasta {diretorio_grupo}.")
        return

    img1 = cv2.imread(imagens_grupo[0])
    img2 = cv2.imread(imagens_grupo[1])

    # Converte para escala de cinza
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    # Calcula o fluxo óptico
    flow = cv2.calcOpticalFlowFarneback(prev=img1, next=img2, flow=None, pyr_scale=0.5, levels=2, winsize=30, iterations=5, poly_n=5, poly_sigma=1.0, flags=0)

    # Cria a imagem do futuro com base no fluxo óptico
    new_image = np.zeros_like(img2)
    for i in range(img2.shape[0]):
        for j in range(img2.shape[1]):
            if img2[i,j] == 255:
                dx = flow[i, j, 0]
                dy = flow[i, j, 1]
                new_i = int(i + dy)
                new_j = int(j + dx)
                if 0 <= new_i < img2.shape[0] and 0 <= new_j < img2.shape[1]:
                    new_image[new_i, new_j] = 255

    # Plotar as imagens suavizadas com diferentes ksizes
    plt.figure(figsize=(10, 10))
    for idx, ksize in enumerate(ksizes):
        ax = plt.subplot(2, 2, idx + 1)
        suavizada = cv2.medianBlur(new_image, ksize)
        ax.imshow(suavizada, cmap='gray')
        ax.set_title(f'ksize={ksize}')
        ax.axis('off')

    #plt.suptitle(f'Análise de Suavização para {os.path.basename(diretorio_grupo)}')
    plt.tight_layout()
    plt.show()

# Itera sobre as pastas dos grupos e aplica Farneback e suavização para cada uma
for pasta_grupo in pastas:
    diretorio_grupo = os.path.join(diretorio_principal, pasta_grupo)
    plotar_fluxo_optico(diretorio_grupo)

print("Processo concluído!")


In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

# Diretório onde estão as pastas dos grupos
diretorio_principal = '2020/2020/1_min_rbr/Marco'
pastas = ['grupo_1', 'grupo_2', 'grupo_3', 'grupo_4', 'grupo_5', 'grupo_6', 'grupo_7', 'grupo_8', 'grupo_9', 'grupo_10']

# Iterar sobre cada diretório de grupo
for pasta_grupo in pastas:
    diretorio_grupo = os.path.join(diretorio_principal, pasta_grupo)

    # Listar as imagens no diretório do grupo
    imagens_grupo = [os.path.join(diretorio_grupo, arquivo) for arquivo in os.listdir(diretorio_grupo) if arquivo.endswith('.jpg')]

    # Verificar se há pelo menos 4 imagens no diretório
    if len(imagens_grupo) < 4:
        print(f'Não há imagens suficientes no grupo {pasta_grupo}.')
        continue

    # Ler as quatro imagens do grupo
    imagens = [cv2.imread(imagem, cv2.IMREAD_GRAYSCALE) for imagem in imagens_grupo]

    # Criar uma nova figura 2x2
    plt.figure(figsize=(10,10))

    # Plotar cada imagem com título adequado
    for i in range(4):
        plt.subplot(2, 2, i+1)
        if i == 0:
            plt.title('Imagem t-1')
        elif i == 1:
            plt.title('Imagem t0')
        elif i == 2:
            plt.title('Imagem t+1')
        else:
            plt.title('Imagem t+1 - Estimado')
        plt.imshow(imagens[i], cmap='gray')
        plt.axis('off')
     # Ajusta o espaçamento entre as imagens
    plt.tight_layout(pad=3.0)
    # Salvar a figura no diretório do grupo com o nome especificado
    plt.savefig(os.path.join(diretorio_grupo, f'{pasta_grupo}_estimacao.png'))
    plt.close()

print('Processo concluído!')


In [ ]:
import cv2
import numpy as np
import os
import pandas as pd

# Função para converter um numpy array em um DataFrame do pandas e salvar em um arquivo .csv
def numpy_para_dataframe_e_csv(matriz_numpy, nome_arquivo_csv):
    # Converter a matriz numpy em DataFrame
    df = pd.DataFrame(matriz_numpy)
    # Salvar o DataFrame em um arquivo .csv
    df.to_csv(nome_arquivo_csv, index=False)

# Diretório onde estão as pastas dos grupos
diretorio_principal = '2020/2020/1_min_rbr/Marco'
pastas = ['grupo_1', 'grupo_2', 'grupo_3', 'grupo_4', 'grupo_5', 'grupo_6', 'grupo_7', 'grupo_8', 'grupo_9', 'grupo_10']

# Função para converter as informações da imagem em DataFrame e salvar em um arquivo .csv
def imagem_para_dataframe_e_csv(caminho_imagem, nome_arquivo_csv):
    # Carregar a imagem
    imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
    # Converter a imagem em um DataFrame
    numpy_para_dataframe_e_csv(imagem, nome_arquivo_csv)

# Itera sobre as pastas dos grupos
for pasta_grupo in pastas:
    diretorio_grupo = os.path.join(diretorio_principal, pasta_grupo)
    # Lista os arquivos na pasta do grupo
    arquivos_grupo = os.listdir(diretorio_grupo)
    # Itera sobre os arquivos na pasta do grupo
    for arquivo in arquivos_grupo:
        if arquivo.endswith('.jpg'):
            caminho_imagem = os.path.join(diretorio_grupo, arquivo)
            nome_arquivo_csv = os.path.join(diretorio_grupo, os.path.splitext(arquivo)[0] + '.csv')
            imagem_para_dataframe_e_csv(caminho_imagem, nome_arquivo_csv)

print("Processo concluído!")


Estimado x Real

In [ ]:
import os
import csv
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

# Função para calcular as métricas de avaliação
def calcular_metricas(imagem_pred, imagem_gt):
    rmse = np.sqrt(mean_squared_error(imagem_pred, imagem_gt)) /255
    mae = mean_absolute_error(imagem_pred, imagem_gt) /255
    bias = np.mean(imagem_pred - imagem_gt)/255
    corr, _ = pearsonr(imagem_pred.ravel(),imagem_gt.ravel())
    return rmse, mae, bias, corr

# Diretório onde estão os arquivos .csv
diretorio_principal = '2020/2020/1_min_rbr/Marco'
pastas = ['grupo_1', 'grupo_2', 'grupo_3', 'grupo_4', 'grupo_5', 'grupo_6', 'grupo_7', 'grupo_8', 'grupo_9', 'grupo_10']

# Listas para armazenar as métricas calculadas
rmse_list = []
mae_list = []
bias_list = []
corr_list = []

# Caminho para o arquivo CSV de métricas
csv_file = os.path.join(diretorio_principal, 'metricas.csv')

# Cabeçalho do arquivo CSV
csv_header = ['Grupo', 'RMSE', 'MAE', 'Bias', 'Correlação de Pearson']

# Abre o arquivo CSV para escrita, criando-o se ainda não existir
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_header)

    # Iterar sobre cada diretório
    for pasta_grupo in pastas:
        diretorio_grupo = os.path.join(diretorio_principal, pasta_grupo)

        # Lista os arquivos na pasta do grupo
        arquivos_grupo = os.listdir(diretorio_grupo)

        # Seleciona o terceiro arquivo da pasta como a imagem original
        if len(arquivos_grupo) < 5:
            print(f"Não foi possível encontrar três arquivos na pasta {diretorio_grupo}.")
            continue

        arquivo_original = os.path.join(diretorio_grupo, arquivos_grupo[4])
        print(arquivo_original)

        # Caminho para o arquivo de imagem suavizada
        arquivo_suavizada = os.path.join(diretorio_grupo, f'{pasta_grupo}_suavizada.csv')
        print(arquivo_suavizada)

        # Carregar os dados dos arquivos CSV como matrizes numpy
        try:
            imagem_gt = np.loadtxt(arquivo_original, delimiter=',')
            imagem_pred = np.loadtxt(arquivo_suavizada, delimiter=',')
        except Exception as e:
            print(f"Erro ao carregar os arquivos CSV para {pasta_grupo}: {e}")
            continue

        # Calcular as métricas de avaliação
        rmse, mae, bias, corr = calcular_metricas(imagem_pred, imagem_gt)

        # Adicionar as métricas às listas
        rmse_list.append(rmse)
        mae_list.append(mae)
        bias_list.append(bias)
        corr_list.append(corr)

        # Escrever as métricas no arquivo CSV
        writer.writerow([pasta_grupo, rmse, mae, bias, corr])

        # Exibir os resultados
        print(f"Métricas para {pasta_grupo}:")
        print(f"RMSE: {rmse}")
        print(f"MAE: {mae}")
        print(f"Viés: {bias}")
        print(f"Correlação de Pearson: {corr}")
        print()

# Exibir médias das métricas
print("Médias das métricas:")
print(f"Média do RMSE: {np.mean(rmse_list)}")
print(f"Média do MAE: {np.mean(mae_list)}")
print(f"Média do Viés: {np.mean(bias_list)}")
print(f"Média da Correlação de Pearson: {np.mean(corr_list)}")

print("Processo concluído!")


2020/2020/1_min_rbr/Marco\grupo_1\defisheye_20200304102700_11_binaria.csv
2020/2020/1_min_rbr/Marco\grupo_1\grupo_1_suavizada.csv
Métricas para grupo_1:
RMSE: 0.47150938088831384
MAE: 0.2226751848334758
Viés: -0.0360456822001735
Correlação de Pearson: 0.5669605801096427

2020/2020/1_min_rbr/Marco\grupo_2\defisheye_20200304143200_11_binaria.csv
2020/2020/1_min_rbr/Marco\grupo_2\grupo_2_suavizada.csv
Métricas para grupo_2:
RMSE: 0.4961212550678129
MAE: 0.2464282365990747
Viés: 0.04692544582488167
Correlação de Pearson: 0.5178329340087936

2020/2020/1_min_rbr/Marco\grupo_3\defisheye_20200305131200_11_binaria.csv
2020/2020/1_min_rbr/Marco\grupo_3\grupo_3_suavizada.csv
Métricas para grupo_3:
RMSE: 0.32618982777265243
MAE: 0.10676335009578544
Viés: -0.03293688785289652
Correlação de Pearson: 0.7905758907670669

2020/2020/1_min_rbr/Marco\grupo_4\defisheye_20200307121600_11_binaria.csv
2020/2020/1_min_rbr/Marco\grupo_4\grupo_4_suavizada.csv
Métricas para grupo_4:
RMSE: 0.43922020457332706
MAE:

In [ ]:
import os
import csv
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

# Função para calcular as métricas de avaliação
def calcular_metricas(imagem_pred, imagem_gt):
    rmse = np.sqrt(mean_squared_error(imagem_pred, imagem_gt)) /255
    mae = mean_absolute_error(imagem_pred, imagem_gt) /255
    bias = np.mean(imagem_pred - imagem_gt)/255
    corr, _ = pearsonr(imagem_pred.ravel(),imagem_gt.ravel())
    return rmse, mae, bias, corr

# Diretório onde estão os arquivos .csv
diretorio_principal = '2020/2020/1_min_rbr/Marco'
pastas = ['grupo_1', 'grupo_2', 'grupo_3', 'grupo_4', 'grupo_5', 'grupo_6', 'grupo_7', 'grupo_8', 'grupo_9', 'grupo_10']

# Listas para armazenar as métricas calculadas
rmse_list = []
mae_list = []
bias_list = []
corr_list = []

# Caminho para o arquivo CSV de métricas
csv_file = os.path.join(diretorio_principal, 'metricas_REAIS.csv')

# Cabeçalho do arquivo CSV
csv_header = ['Grupo', 'RMSE', 'MAE', 'Bias', 'Correlação de Pearson']

# Abre o arquivo CSV para escrita, criando-o se ainda não existir
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_header)

    # Iterar sobre cada diretório
    for pasta_grupo in pastas:
        diretorio_grupo = os.path.join(diretorio_principal, pasta_grupo)

        # Lista os arquivos na pasta do grupo
        arquivos_grupo = os.listdir(diretorio_grupo)

        # Verifica se há pelo menos 5 arquivos na pasta
        if len(arquivos_grupo) < 5:
            print(f"Não foi possível encontrar cinco arquivos na pasta {diretorio_grupo}.")
            continue

        # Seleciona o terceiro arquivo da pasta como a imagem original
        arquivo_original = os.path.join(diretorio_grupo, arquivos_grupo[2])
        print(f"Arquivo original: {arquivo_original}")

        # Seleciona o quinto arquivo da pasta como a imagem predita
        arquivo_predito = os.path.join(diretorio_grupo, arquivos_grupo[4])
        print(f"Arquivo predito: {arquivo_predito}")

        # Carregar os dados dos arquivos CSV como matrizes numpy
        try:
            imagem_gt = np.loadtxt(arquivo_original, delimiter=',')
            imagem_pred = np.loadtxt(arquivo_predito, delimiter=',')
        except Exception as e:
            print(f"Erro ao carregar os arquivos CSV para {pasta_grupo}: {e}")
            continue

        # Calcular as métricas de avaliação
        rmse, mae, bias, corr = calcular_metricas(imagem_pred, imagem_gt)

        # Adicionar as métricas às listas
        rmse_list.append(rmse)
        mae_list.append(mae)
        bias_list.append(bias)
        corr_list.append(corr)

        # Escrever as métricas no arquivo CSV
        writer.writerow([pasta_grupo, rmse, mae, bias, corr])

        # Exibir os resultados
        print(f"Métricas para {pasta_grupo}:")
        print(f"RMSE: {rmse}")
        print(f"MAE: {mae}")
        print(f"Viés: {bias}")
        print(f"Correlação de Pearson: {corr}")
        print()

# Exibir médias das métricas
print("Médias das métricas:")
print(f"Média do RMSE: {np.mean(rmse_list)}")
print(f"Média do MAE: {np.mean(mae_list)}")
print(f"Média do Viés: {np.mean(bias_list)}")
print(f"Média da Correlação de Pearson: {np.mean(corr_list)}")

print("Processo concluído!")


Arquivo original: 2020/2020/1_min_rbr/Marco\grupo_1\defisheye_20200304102600_11_binaria.csv
Arquivo predito: 2020/2020/1_min_rbr/Marco\grupo_1\defisheye_20200304102700_11_binaria.csv
Métricas para grupo_1:
RMSE: 0.46588800296085914
MAE: 0.21736824884068787
Viés: 0.012562007053146567
Correlação de Pearson: 0.5747700430888769

Arquivo original: 2020/2020/1_min_rbr/Marco\grupo_2\defisheye_20200304143100_11_binaria.csv
Arquivo predito: 2020/2020/1_min_rbr/Marco\grupo_2\defisheye_20200304143200_11_binaria.csv
Métricas para grupo_2:
RMSE: 0.4966445681411501
MAE: 0.24692800391274486
Viés: -0.06637617585664715
Correlação de Pearson: 0.521714800360563

Arquivo original: 2020/2020/1_min_rbr/Marco\grupo_3\defisheye_20200305131100_11_binaria.csv
Arquivo predito: 2020/2020/1_min_rbr/Marco\grupo_3\defisheye_20200305131200_11_binaria.csv
Métricas para grupo_3:
RMSE: 0.3168189861710626
MAE: 0.10076129359247495
Viés: 0.008311313844557558
Correlação de Pearson: 0.8018170429843052

Arquivo original: 2020